# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [11]:
# your code here
orders = pd.read_csv('Orders.csv', index_col=0)
orders
#orders.shape


,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [24]:
# your code here
order_amount=pd.DataFrame(orders.groupby(['CustomerID','Country'])['amount_spent'].agg(sum)).reset_index()
order_amount['Customer_type'] = pd.qcut(order_amount['amount_spent'], q=[0, 0.75, 0.95, 1], labels=['Non-preferred', 'Preferred', 'VIP'])
order_amount

,CustomerID,Country,amount_spent,Customer_type
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
4,12350,Norway,334.40,Non-preferred
...,...,...,...,...
4342,18280,United Kingdom,180.60,Non-preferred
4343,18281,United Kingdom,80.82,Non-preferred
4344,18282,United Kingdom,178.05,Non-preferred
4345,18283,United Kingdom,2094.88,Preferred


In [25]:
order_amount['Customer_type'].value_counts()

Non-preferred    3260
Preferred         869
VIP               218
Name: Customer_type, dtype: int64

In [26]:
VIP = order_amount[order_amount['Customer_type'] == 'VIP']
VIP

,CustomerID,Country,amount_spent,Customer_type
0,12346,United Kingdom,77183.60,VIP
10,12357,Switzerland,6207.67,VIP
12,12359,Cyprus,6372.58,VIP
52,12409,Switzerland,11072.67,VIP
57,12415,Australia,124914.53,VIP
...,...,...,...,...
4215,18109,United Kingdom,8052.97,VIP
4237,18139,United Kingdom,8438.34,VIP
4261,18172,United Kingdom,7561.68,VIP
4300,18223,United Kingdom,6484.54,VIP


In [27]:
Preferred = order_amount[order_amount['Customer_type'] == 'Preferred']
Preferred

,CustomerID,Country,amount_spent,Customer_type
1,12347,Iceland,4310.00,Preferred
2,12348,Finland,1797.24,Preferred
3,12349,Italy,1757.55,Preferred
5,12352,Norway,2506.04,Preferred
9,12356,Portugal,2811.43,Preferred
...,...,...,...,...
4327,18259,United Kingdom,2338.60,Preferred
4328,18260,United Kingdom,2643.20,Preferred
4336,18272,United Kingdom,3078.58,Preferred
4345,18283,United Kingdom,2094.88,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [29]:
# your code here
VIP['Country'].value_counts()

United Kingdom     178
Germany             11
France               9
Switzerland          3
Spain                2
Japan                2
Portugal             2
EIRE                 2
Australia            1
Finland              1
Norway               1
Denmark              1
Cyprus               1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [31]:
# your code here
order_amount[(order_amount['Customer_type'] == 'Preferred')|(order_amount['Customer_type'] =='VIP')]['Country'].value_counts()

United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Spain                7
Portugal             7
Norway               7
Italy                5
Finland              5
Channel Islands      4
Australia            4
Japan                4
Cyprus               3
Denmark              3
EIRE                 3
Israel               2
Sweden               2
Singapore            1
Lebanon              1
Poland               1
Iceland              1
Greece               1
Netherlands          1
Austria              1
Canada               1
Malta                1
Name: Country, dtype: int64